### Calculate graph vectors $\vec{G}=\left[S_{mean},\sigma_S,Q,P_{mean},C_{mean},T,A,E^{glo},E^{loc}_{mean},EC_{mean}\right]$

In [43]:
def calc_graph_vector( filename, thresholds ) :
    #-------------------------------------------------------------------#   
    #    Input arguments:                                               #
    #        filename   – name of file containing connectivity matrix   #
    #        thresholds – list containing thresholds of interest        #
    #-------------------------------------------------------------------#
    #=== import libraries
    import numpy as np
    import bct

    #=== load matrix 
    A_raw = np.loadtxt(filename)
    N = A_raw.shape[0]   # number of nodes
    M_sat = N*(N-1)/2    # max number of connections 

    #=== parameters
    N_rep_louvain = 10   # number of Louvain algorithm repetitions
    N_measures = 10      # number of graph measures
    gamma = 1            # Louvain resolution parameter

    graph_measures = np.zeros([ len(thresholds), N_measures ])      # create empty output matrix
    for thr in range(len(thresholds)) : 
        #=== thresholding 
        A = bct.threshold_proportional( A_raw, p=thresholds[thr], copy=True );
        A[np.nonzero(A<0)] = 0                                      # ensure only positive weights
        M_act = A[np.nonzero(A>0)].shape[0] / 2                     # actual number of nonzero connections
        #=== calculate measures
        #-- mean connection strenght 
        S = np.sum(A)/M_act
        #-- connection strenght std
        Svar = np.std(A[np.nonzero(A)])
        #-- modularity
        [M,Q] = bct.modularity_louvain_und(A, gamma)
        for i in range(N_rep_louvain) :
            [Mt,Qt] = bct.modularity_louvain_und(A, gamma)
            if Qt > Q :
                Q = Qt
                M = Mt
        #-- participation coefficient
        P = np.mean(bct.participation_coef_sign(A, M))
        #-- clustering 
        C = np.mean(bct.clustering_coef_wu(A))
        #-- transitivity 
        T = bct.transitivity_wu(A)
        #-- assortativity
        Asso = bct.assortativity_wei(A)
        #-- global & local efficiency 
        Eglo = bct.efficiency_wei(A)
        Eloc = np.mean(bct.efficiency_wei(A, local=True))
        #-- mean eigenvector centralit
        Eig = np.mean(bct.eigenvector_centrality_und(A))
        #=== write vector to matrix
        graph_measures[thr] = [ S, Svar, Q, P, C, T, Asso, Eglo, Eloc, Eig ]

    #=== save results to file
    np.savetxt( filename[:-4]+'_GV.txt', graph_measures )

### Find out subjects with all files ( 4 sessions $\times$ 5 atlases $\times$ 4 models )

In [44]:
import os
import numpy as np
#=== loop over files
sub_counter = np.zeros(57)  # stores number of files per subjects
for sub in range(1,57) :
    for ses in range(1,5) :
        for model in ['cor','par','cov','pre'] :
            for atlas in ['pow','aal','dos','har','mul'] :
                filename = 'data/matKFLB_sub' + str(sub).zfill(2) + '_ses' + str(ses) + '_' + atlas + '_' + model + '.txt'
                if os.path.isfile(filename)==True :               # if file exists
                    sub_counter[sub-1] = sub_counter[sub-1] + 1 
#=== filter complete subjects
sub_complete = np.nonzero(sub_counter==80)[0] + 1

In [ ]:
#=== define thresholds
thresholds = [ 0.05, 0.075, 0.1, 0.125, 0.15 ]
#=== calculate graph measures for all files
for sub in list(sub_complete) :
    for ses in range(1,5) :
        for model in ['cor','par','cov','pre'] :
            for atlas in ['pow','aal','dos','har','mul'] :
                filename = 'data/matKFLB_sub' + str(sub).zfill(2) + '_ses' + str(ses) + '_' + atlas + '_' + model + '.txt'
                #=== do the job
                print('Running calc_graph_vector for ' + filename)
                calc_graph_vector(filename,thresholds)

Running calc_graph_vector for data/matKFLB_sub01_ses1_pow_cor.txt
Running calc_graph_vector for data/matKFLB_sub01_ses1_aal_cor.txt
Running calc_graph_vector for data/matKFLB_sub01_ses1_dos_cor.txt
Running calc_graph_vector for data/matKFLB_sub01_ses1_har_cor.txt
Running calc_graph_vector for data/matKFLB_sub01_ses1_mul_cor.txt
